In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
import pyarrow.parquet as pq
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, auc,
    accuracy_score, f1_score, precision_score, recall_score
)
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings('ignore')

def train_and_evaluate_model(X_train_path, y_train_path, X_test_path, y_test_path):
    """Entrena y evalúa un modelo Random Forest con Grid Search"""
    start_time = datetime.now()
    process = psutil.Process(os.getpid())
    mem_before = process.memory_info().rss / (1024 ** 3)
    
    # Inicializar mem_tracking
    mem_tracking = [mem_before]  # Incluir la memoria inicial
    
    try:
        # 1. Cargar datos de entrenamiento
        print("Cargando datos de entrenamiento...")
        X_train = pq.read_table(X_train_path).to_pandas()
        y_train = pq.read_table(y_train_path).to_pandas().squeeze()

        # Actualizar memoria
        mem_tracking.append(process.memory_info().rss / (1024 ** 3))

        # 2. Cargar datos de prueba
        print("Cargando datos de prueba...")
        X_test = pq.read_table(X_test_path).to_pandas()
        y_test = pq.read_table(y_test_path).to_pandas().squeeze()

        # Actualizar memoria
        mem_tracking.append(process.memory_info().rss / (1024 ** 3))

        # 3. Verificar que las columnas de entrenamiento y prueba coincidan
        missing_columns = set(X_train.columns) - set(X_test.columns)
        if missing_columns:
            print(f"Advertencia: Columnas faltantes en X_test: {missing_columns}. Rellenando con 0.")
            for col in missing_columns:
                X_test[col] = 0  # Rellenar con 0 o NaN según sea necesario

        # 4. Selección de características
        print("Selección inicial de características...")
        selector = SelectKBest(f_classif, k=min(100, X_train.shape[1]))
        selector.fit(X_train, y_train)
        selected_features = selector.get_support(indices=True)

        # Actualizar memoria
        mem_tracking.append(process.memory_info().rss / (1024 ** 3))

        # 5. Configuración del modelo Random Forest
        model = RandomForestClassifier(
            class_weight='balanced',  # Balancear clases
            random_state=42
        )

        # 6. Definir la cuadrícula de hiperparámetros para Grid Search
        param_grid = {
            'n_estimators': [100, 200],  # Número de árboles
            'max_depth': [None, 10, 20],  # Profundidad máxima de los árboles
            'min_samples_split': [2, 5],  # Mínimo de muestras para dividir un nodo
            'min_samples_leaf': [1, 2],  # Mínimo de muestras en una hoja
            'max_features': ['sqrt', 'log2']  # Número de características a considerar en cada división
        }

        # 7. Configurar Grid Search
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring='f1_macro',  # Métrica a optimizar
            cv=3,  # Validación cruzada de 3 folds
            n_jobs=-1,  # Usar todos los núcleos disponibles
            verbose=2
        )

        # 8. Aplicar SMOTE solo en el conjunto de entrenamiento
        smote = SMOTE(sampling_strategy='auto', k_neighbors=3, random_state=42)
        X_res, y_res = smote.fit_resample(X_train.iloc[:, selected_features], y_train)

        # Actualizar memoria
        mem_tracking.append(process.memory_info().rss / (1024 ** 3))

        # 9. Entrenar el modelo con Grid Search
        print("Iniciando Grid Search...")
        grid_search.fit(X_res, y_res)

        # Actualizar memoria
        mem_tracking.append(process.memory_info().rss / (1024 ** 3))

        # 10. Mejor modelo encontrado
        best_model = grid_search.best_estimator_
        print(f"Mejores hiperparámetros: {grid_search.best_params_}")

        # 11. Evaluación final
        y_pred = best_model.predict(X_test.iloc[:, selected_features])
        y_proba = best_model.predict_proba(X_test.iloc[:, selected_features])
        
        # Verificar que y_proba no contenga NaN o infinitos
        if np.isnan(y_proba).any() or np.isinf(y_proba).any():
            raise ValueError("Las probabilidades predichas contienen NaN o infinitos. Revisa el entrenamiento del modelo.")
        
        # Verificar la distribución de clases en el conjunto de prueba
        class_distribution_test = y_test.value_counts()
        print("Distribución de clases en el conjunto de prueba:", class_distribution_test.to_dict())
        
        # Filtrar clases con menos de 2 muestras
        valid_classes = class_distribution_test[class_distribution_test >= 2].index
        
        # Calcular la matriz de confusión
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        # Generar curvas ROC para todas las clases
        plt.figure(figsize=(10, 8))
        auc_scores = {}
        for i in valid_classes:
            try:
                # Ajustar el índice para coincidir con la columna de y_proba
                col_idx = i - 1  # Restar 1 para convertir de índice 1-5 a índice 0-4
                fpr, tpr, _ = roc_curve((y_test == i).astype(int), y_proba[:, col_idx])
                auc_value = auc(fpr, tpr)
                auc_scores[f"Clase {i}"] = auc_value
                plt.plot(fpr, tpr, label=f'Clase {i} (AUC = {auc_value:.2f})')
            except ValueError as e:
                print(f"Error calculando AUC para la clase {i}: {str(e)}")
                auc_scores[f"Clase {i}"] = np.nan

        # Verificar si se pudo calcular al menos un AUC válido
        if not auc_scores or all(np.isnan(value) for value in auc_scores.values()):
            print("No se pudo calcular el AUC para ninguna clase. Generando gráfico vacío.")
            plt.text(0.5, 0.5, 'No se pudo calcular el AUC para ninguna clase', ha='center', va='center')
        
        plt.title('Curvas ROC')
        plt.xlabel('Tasa de Falsos Positivos')
        plt.ylabel('Tasa de Verdaderos Positivos')
        plt.legend()
        roc_plot_path = os.path.join("graficos", f"roc_curve_{os.path.basename(X_train_path)}.png")
        plt.savefig(roc_plot_path)
        plt.close()

        # Generar matriz de confusión detallada
        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
        plt.title('Matriz de Confusión')
        conf_matrix_path = os.path.join("graficos", f"conf_matrix_{os.path.basename(X_train_path)}.png")
        plt.savefig(conf_matrix_path)
        plt.close()

        # Resultados finales
        end_time = datetime.now()
        return {
            'model_name': 'RandomForest-Optimizado',
            'dataset': os.path.basename(X_train_path),
            'start_time': start_time,
            'end_time': end_time,
            'duration': (end_time - start_time).total_seconds() / 60,
            'avg_memory': np.mean(mem_tracking),
            'test_accuracy': accuracy_score(y_test, y_pred),
            'test_precision': precision_score(y_test, y_pred, average='macro'),
            'test_recall': recall_score(y_test, y_pred, average='macro'),
            'test_f1': f1_score(y_test, y_pred, average='macro'),
            'classification_report': classification_report(y_test, y_pred, output_dict=True, zero_division=0),
            'conf_matrix_path': conf_matrix_path,
            'roc_curve_path': roc_plot_path,
            'auc_scores': auc_scores,
            'selected_features': len(selected_features),
            'best_params': grid_search.best_params_
        }

    except Exception as e:
        print(f"Error procesando {X_train_path}: {str(e)}")
        return None

def main():
    input_root = r"C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\train_test_splits_20250224_221733"
    output_root = os.path.join(input_root, "reportes_modelos")
    os.makedirs(output_root, exist_ok=True)
    
    train_files = []
    for root, _, files in os.walk(input_root):
        for file in files:
            if file.startswith("X_train") and file.endswith(".parquet"):
                base = file.replace("X_train_", "").replace(".parquet", "")
                required = {
                    'X_train': file,
                    'y_train': f"y_train_{base}.parquet",
                    'X_test': f"X_test_{base}.parquet",
                    'y_test': f"y_test_{base}.parquet"
                }
                
                paths = {}
                for k, v in required.items():
                    full_path = os.path.join(root, v)
                    if os.path.exists(full_path):
                        paths[k] = full_path
                    else:
                        break
                
                if len(paths) == 4:
                    train_files.append(paths)
    
    results = []
    for dataset in train_files:
        print(f"\nProcesando: {dataset['X_train']}")
        result = train_and_evaluate_model(
            dataset['X_train'],
            dataset['y_train'],
            dataset['X_test'],
            dataset['y_test']
        )
        if result:
            results.append(result)
            print(f"✅ Procesado correctamente")
    
    if results:
        generate_pdf_report(results, output_root)
    else:
        print("\n⚠️ No se encontraron resultados válidos")

if __name__ == "__main__":
    main()


Procesando: C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\train_test_splits_20250224_221733\MaxAbs_full_full\X_train_MaxAbs_full_full.parquet
Calculando medianas...
Selección inicial de características...
Iniciando Grid Search...
Fitting 3 folds for each of 48 candidates, totalling 144 fits
Mejores hiperparámetros: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Distribución de clases en el conjunto de prueba: {4: 31091, 1: 31091, 5: 31090, 3: 31090, 2: 31090}
✅ Procesado correctamente

Procesando: C:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\outputs\experiments\train_test_splits_20250224_221733\MaxAbs_Linear_selected\X_train_MaxAbs_Linear_selected.parquet
Calculando medianas...
Selección inicial de características...
Iniciando Grid Search...
Fitting 3 folds for each of 48 candidates, totalling 144 fits
Mejores hiperparámetros: {'max_depth': None, 'max_features': 